# 🏥 Adaptive Reasoning for Healthcare Administration

## Real-World Multi-Task Reasoning with Symbolic AI

This demo showcases an **adaptive LLM-symbolic reasoning framework** solving realistic healthcare administration tasks, automatically routing each problem to the optimal solver.

---

### 📋 Scenario: Hospital Admin Assistant

A hospital administrator faces three different reasoning tasks in a single workflow:

<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 15px; margin-top: 20px;">

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">📊 Task 1: Clinical Trial Matching</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> Constraint Satisfaction</p>
<p style="font-size: 0.85em; opacity: 0.95;">Match patient profile against complex trial inclusion/exclusion criteria</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">⚙️ Optimal solver: Z3 (SMT)</p>
</div>

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">🔐 Task 2: Policy Compliance</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> First-Order Logic</p>
<p style="font-size: 0.85em; opacity: 0.95;">Verify staff credentialing against hospital policies</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">⚙️ Optimal solver: Prover9</p>
</div>

<div style="background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); padding: 20px; border-radius: 10px; color: white; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
<h4 style="margin-top: 0; color: white;">✅ Task 3: Discharge Readiness</h4>
<p style="font-size: 0.9em; margin-bottom: 5px;"><strong>Type:</strong> Logic Programming</p>
<p style="font-size: 0.85em; opacity: 0.95;">Evaluate patient discharge eligibility via forward chaining</p>
<p style="font-size: 0.8em; margin-top: 10px; opacity: 0.9;">⚙️ Optimal solver: Pyke</p>
</div>

</div>

---

### 🎯 Framework Capabilities

✨ **Automatic Problem Detection** - Identifies reasoning type without manual specification  
🧠 **Heterogeneous Solver Selection** - Routes to optimal symbolic solver (Z3, Prover9, Pyke)  
⚡ **Parallel Execution** - Processes independent tasks concurrently  
🔄 **End-to-End Automation** - From natural language to verified answers


## Step 1: Load Hospital Admin Tasks

Loading three realistic healthcare administration tasks that would typically be handled by different software systems.


In [4]:
problem_description = '''
PATIENT CASE FOR TRIAL MATCHING:
Patient Profile:
- 62-year-old male
- History: CVA, neurogenic bladder with indwelling suprapubic catheter
- Multiple prior admissions for UTIs, altered mental status, and urosepsis
- Current: Urosepsis (now resolved after vanc/meropenem treatment)
- CT findings: Non-obstructing stone in left ureter, no urethral strictures
- Medical history: Non-Hodgkins Marginal Zone Lymphoma (s/p R-CHOP x6), Bell's Palsy, BPH, Hypertension, Partial bowel obstruction s/p colostomy, Hepatitis C, Cryoglobulinemia, SLE with transverse myelitis (anti-dsDNA Ab+), Insulin-dependent diabetes, Fungal esophagitis, Recurrent UTIs (Pseudomonas & Enterococcus)

Clinical Trial: NCT02142751
Trial Criteria:
Inclusion:
- ≥18 years old hospitalized patients
- Negative pregnancy test in fertile women
- Episode of monomicrobial urinary BSI due to multidrug-resistant E.coli (susceptible to fosfomycin and meropenem or ceftriaxone)
- Urinary sepsis with MDR E.coli from blood cultures, requires:
  Clinical criteria (at least one): UTI symptoms (dysuria, urgency, suprapubic pain, pollakiuria), lumbar back pain, costovertebral angle tenderness, altered mental status (≤70 years), intermittent/permanent indwelling Foley catheter
  Urinalysis criteria (at least one): Positive dipstick for nitrites or leukocyte esterase, positive urine culture

Exclusion criteria:
- Polymicrobial bacteremia
- Undrained renal abscess or unresolved obstructive uropathy
- Pregnant or breastfeeding women
- Hematogenous infection
- Other concomitant infection
- Renal transplant recipients
- Polycystic kidney
- Hypersensitivity to meropenem, fosfomycin, or ceftriaxone

Based on the available information, can this patient be enrolled in the trial?
A) Yes 
B) No


HOSPITAL CREDENTIALING VERIFICATION:
Excerpt from Hospital Policy Manual Section - Staff Credentialing Requirements:
- All hospital staff members shall be classified as either clinical staff or administrative staff. 
- Clinical staff are defined as personnel who have direct patient contact in the course of their duties.
- All clinical staff are required to complete annual infection control training as mandated by Joint Commission standards. 
- Upon successful completion of annual infection control training, staff members receive credentialing authorization. 
- Nurses are classified as clinical staff. Physicians are classified as clinical staff. 
- Billing clerks are classified as administrative staff. 
- Administrative staff do not have patient contact.

Personnel Record for Johnson, R.N.:
- Position: Registered Nurse
- Annual Infection Control Training Status: Not Completed

Question: According to hospital policy, is Nurse Johnson currently credentialed?
A) Yes
B) No


DISCHARGE READINESS VERIFICATION:
Patient Discharge Protocol - Standard Operating Procedure:
- If a patient has stable vital signs, then the patient meets vital stability criteria. 
- If a patient is off IV antibiotics, then the patient meets antibiotic transition criteria. 
- If a patient meets vital stability criteria and meets antibiotic transition criteria, then the patient is medically stable. 
- If a patient can perform self-care activities, then the patient is functionally independent. 
- If a patient is functionally independent and has completed discharge teaching, then the patient meets discharge education requirements. 
- If a patient is medically stable and meets discharge education requirements, then the patient is cleared for discharge.

Patient Status Report - Chen, Margaret (MRN: 445821):
Margaret Chen has stable vital signs. Margaret Chen is off IV antibiotics. Margaret Chen can perform self-care activities. Margaret Chen has completed discharge teaching.

Based on this discharge protocol, is Margaret Chen cleared for discharge?
A) Yes
B) No
'''


## Step 2: Devise Plan for Adaptive Reasoning 

The **Carnap Router** analyzes tasks and creates an execution plan.

Let's first initialize load the config and initialize an LLM:


In [9]:
import yaml
from agents.generation.api import AzureOpenAIGenerator
from agents.meta_agents.planner import Planner, TracePersister

# Load config and initialize
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

azure_config = config['api_config']['gpt-4o-azure']
llm = AzureOpenAIGenerator(
    model_name=azure_config['model_name'],
    api_key=azure_config['api_key'],
    model_version=azure_config['openai_api_version'],
    azure_endpoint=azure_config['azure_endpoint']
)

Now, the router:

- Parses the multi-task input
- Classifies each task by reasoning type
- Constructs an execution DAG
- Routes tasks to appropriate symbolic solvers

In [11]:
router = Planner(generator=llm)
plan, memory, problem_ids = router({"problem": problem_description})

2025-11-06 13:48:53,614 | INFO | Registered agent '<PLAN_START>'
2025-11-06 13:48:53,615 | INFO | Registered agent '<PLAN_END>'
2025-11-06 13:48:53,615 | INFO | Registered agent 'lp_solver'
2025-11-06 13:48:53,615 | INFO | Registered agent 'fol_solver'
2025-11-06 13:48:53,616 | INFO | Registered agent 'csp_solver'
2025-11-06 13:48:53,616 | INFO | Registered agent 'smt_solver'
2025-11-06 13:48:53,617 | INFO | Registered agent 'ilp_solver'
2025-11-06 13:48:53,617 | INFO | Registered agent 'epistemic_solver'
2025-11-06 13:48:53,618 | INFO | Registered agent 'risk_solver'
2025-11-06 13:48:53,618 | INFO | Registered agent 'compositional_solver'
2025-11-06 13:48:53,619 | INFO | Registered agent 'causal_solver'
2025-11-06 13:48:59,153 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-06 13:48:59,155 | INFO | Scratchpad WRITE problem_type_ques_1=SAT (ttl=None)
2025-11-06 13:48:

## Step 3: Let CARNAP Solve It

We can now execute the plan!


In [14]:
plan[0].execute(memory, TracePersister())

2025-11-06 13:50:26,136 | INFO | Execute plan with topological order...
2025-11-06 13:50:26,136 | INFO | Trace saved: <PLAN_START> → None
2025-11-06 13:50:26,137 | INFO | Trace saved: ques_1 → None
2025-11-06 13:50:26,137 | INFO | Scratchpad WRITE smt_problem_queue_smt_solver=[] (ttl=None)
2025-11-06 13:50:33,873 | INFO | HTTP Request: POST https://lunarchatgpt.openai.azure.com/openai/deployments/lunar-chatgpt-4o/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
2025-11-06 13:50:33,875 | INFO | Attempt 1/1
2025-11-06 13:50:33,876 | INFO | Running Z3 command: z3 -smt2 /tmp/tmptq1cxqdz.smt2
2025-11-06 13:50:33,890 | INFO | Z3 ran successfully. Found 1 solutions
2025-11-06 13:50:33,892 | INFO | Scratchpad WRITE smt_results_queue=[{'ori_answer': {}, 'parsed_answer': 'A', 'success': True, ' (ttl=None)
2025-11-06 13:50:33,892 | INFO | Scratchpad WRITE result_ques_1={'ori_answer': {}, 'parsed_answer': 'A', 'success': True, 'e (ttl=None)
2025-11-06 13:50:33,893 | INFO | Trace saved: sm

Collecting results from memory:

In [19]:
predictions = []
for pid in problem_ids:
    result = memory.read(f"result_{pid}") or {}
    predictions.append(result.get('parsed_answer', None))

print(predictions)

['A', 'B', 'A']


The three predictions correspond to our three tasks, where **A = Yes** and **B = No**. For example, ['A', 'B', 'A']
 means:

1. **Clinical Trial Matching:** A → Patient **can** be enrolled
2. **Policy Compliance:** B → Nurse Johnson is **not** credentialed
3. **Discharge Readiness:** A → Margaret Chen is **cleared** for discharge